Ultilize the **astrorapid** package to classify light curves, Using the data from ZTF.

**NOTE**: The place 'TBD' should be replaced with proper values according to the exact data being used.

In [ ]:
from astrorapid.classify import Classify
import numpy as np
from ztfquery import lightcurve
import matplotlib.pyplot as plt

In [ ]:
def mag2fluxdensity(mag, magerr, mode = 'muJy'):
    #convert magnitude to flux density
    if mode == 'muJy':
        flux = 10**(-mag/2.5)*3631*1e6
        fluxerr = flux*np.log(10)*magerr/2.5
        return flux,fluxerr
    elif mode == 'cgs':
        #both are correct
        #return 10**(-mag/2.5)*3631*1e-23
        flux = 10**(-(mag+48.6)/2.5)
        fluxerr = flux*np.log(10)*magerr/2.5
        return flux, fluxerr
def fluxdensity2mag(flux, fluxerr, mode = 'muJy'):
    #convert flux density to magnitude
    if mode == 'muJy':
        mag = -2.5*np.log10(flux/3631/1e6)
        magerr = 2.5*fluxerr/np.log(10)/flux
        return mag, magerr
    elif mode == 'cgs':
        #both are correct
        #return -2.5*np.log10(flux/3631*1e23)
        mag = -2.5*np.log10(flux)-48.6
        magerr = 2.5*fluxerr/np.log(10)/flux
        return mag, magerr

In [ ]:
coord = np.array([[125.5451334, 7.7287351, 0.173],
[54.646051, 18.253337, -999]])  # example coordinates(ra, dec, and redshift)

In [ ]:
ra, dec, redshift = coord[0,0], coord[0,1], coord[0,2]
objid = "transient0"

# Get light curve data from ZTF with ztfquery package
lcq = lightcurve.LCQuery.from_position(ra,dec, 3)   # need to set irsa acount for the first time user
data = lcq.data
lcq.show()  # show the light curve

In [ ]:
data.columns    # show column names

In [ ]:
idx = np.where(data['mjd']> TBD)   # Select data just before the flare, TBD shoud be replaced by the actual mjd time
data_idx = data.iloc[idx]
print(data_idx.shape)
data_g = data_idx.iloc[np.where(data_idx['filtercode']=='zg')]
data_r = data_idx.iloc[np.where(data_idx['filtercode']=='zr')]
mag_g = data_g['mag']
mag_r = data_r['mag']
t_g = data_g['mjd'].values
t_r = data_r['mjd'].values
magerr_g = data_g['magerr']
magerr_r = data_r['magerr']
band_g = data_g['filtercode'].values
band_r = data_r['filtercode'].values

flux_g, fluxerr_g = mag2fluxdensity(mag_g, magerr_g)
flux_r, fluxerr_r = mag2fluxdensity(mag_r, magerr_r)

mjd = np.concatenate((t_g, t_r))
flux = np.concatenate((flux_g, flux_r))
fluxerr = np.concatenate((fluxerr_g, fluxerr_r))
passband = np.concatenate((band_g, band_r))
passband = passband.astype(str)
passband = np.where(passband == 'zg', 'g', passband)
passband = np.where(passband == 'zr', 'r', passband)

In [ ]:
# show the data used for classification
plt.figure(figsize=(10,6))
plt.errorbar(t_g,flux_g,yerr=fluxerr_g,fmt='o',label='g band')
plt.errorbar(t_r,flux_r,yerr=fluxerr_r,fmt='o',label='r band')
plt.xlabel('Time')
plt.ylabel('Flux')
plt.legend()
plt.show()

In [ ]:
# the data needed for classification is: mjd, flux, fluxerr, passband, photoflag, ra, dec, objid, redshift, mwebv
print("mjd: ",mjd.shape,mjd)
print("flux: ",flux.shape, flux)
print("fluxerr: ",fluxerr.shape, fluxerr)
print("passband: ",passband)
photoflag = np.full(len(mjd),4096)  #4096 for good detection, 0 for bad detection, 6144 for trigger time
photoflag[TBD] = 6144 # trigger time, set according to the up figure
mwebv = 0.0 # Optional parameter, Milky Way E(B-V), set 0 for None
print("photoflag: ",photoflag)

In [ ]:
# according the known or unknown redshift, choose the model and classify the light curve.
if redshift == -999:
    redshift = None
    known_redshift = False
    model_name = 'ZTF_unknown_redshift'
else:
    known_redshift = True
    model_name = 'ZTF_known_redshift'

light_curve_list = [(mjd, flux, fluxerr, passband, photoflag, ra, dec, objid, redshift, mwebv)]

classification = Classify(model_name=model_name, known_redshift=known_redshift)
predictions, time_steps = classification.get_predictions(light_curve_list)
print(predictions)

save_path = r'./classification_results/'

classification.plot_light_curves_and_classifications(figdir=save_path)
classification.plot_classification_animation(figdir=save_path)